<a href="https://colab.research.google.com/github/KashmiraG/Flood-Prediction/blob/main/Flood_Prediction_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load your dataset (replace 'your_dataset.csv' with your data file)
data = pd.read_csv('/content/IndianWeatherRepository (1).csv')

# Data preprocessing
# Encode 'condition_text' using one-hot encoding
label_encoder = LabelEncoder()
data['condition_text'] = label_encoder.fit_transform(data['condition_text'])
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
data_encoded = onehot_encoder.fit_transform(data[['condition_text']])
data_encoded = pd.DataFrame(data_encoded, columns=['condition_' + str(i) for i in range(data_encoded.shape[1])])

# Combine the one-hot encoded columns with the original data
data = pd.concat([data, data_encoded], axis=1)

# Define your features (X) and target (y)
X = data[['temperature_celsius', 'wind_kph', 'pressure_in', 'humidity', 'cloud', 'Water Level'] + list(data.columns[7:])]
y = data['Flood']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a data preprocessing pipeline (optional)
scaler = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, ['temperature_celsius', 'wind_kph', 'pressure_in', 'humidity', 'cloud'] + list(data.columns[7:]))
    ])

# Create an SVR model
svr = SVR(kernel='linear', C=1.0)

# Create a pipeline that includes data preprocessing and the SVR model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svr', svr)
])

# Train the SVR model using the pipeline
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.010049276769063602
R-squared: 0.7282675561645202


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
